In [26]:
import copy
import collections
import os
import pickle

from petreader.labels import *
import pandas as pd

from PetReader import pet_reader
from labels import *
from utils import read_json_to_dict

In [62]:
from utils import read_keywords

## Load Results

In [58]:
token_cls_goldstandard = read_json_to_dict("data/other/token_goldstandard.json")

keywords_gold_token_cls_results = read_json_to_dict("data/results/key_words_gold/results-token-classification.json")
keywords_gold_token_cls = read_json_to_dict("data/results/key_words_gold/token-classification.json")

keywords_literature_token_cls_results = read_json_to_dict("data/results/key_words_literature/results-token-classification.json")
keywords_literature_token_cls = read_json_to_dict("data/results/key_words_literature/token-classification.json")

keywords_literature_filtered_token_cls_results = read_json_to_dict("data/results/key_words_literature_filtered/results-token-classification.json")
keywords_literature_filtered_token_cls = read_json_to_dict("data/results/key_words_literature_filtered/token-classification.json")

## Debug

In [28]:
from petreader.TokenClassification import TokenClassification
from petbenchmarks.tokenclassification import TokenClassificationBenchmark
benchmark = TokenClassificationBenchmark(pet_reader.token_dataset)

gold = benchmark.GetGoldStandard()
print(gold['doc-3.2'][XOR_GATEWAY])

['If', 'otherwise']


In [29]:
print(pet_reader.token_dataset.GetXORGateways('doc-3.2'))

[[], [['If'], ['otherwise']], [], []]


## 1) Analyze Keyword Results

### a) check for documents with recall != 1 (GOLD)

In [30]:
for doc_name, doc_dict in keywords_gold_token_cls_results.items():
    if doc_name.startswith('doc'):
        if doc_dict[XOR_GATEWAY][RECALL] != 1 and doc_dict[XOR_GATEWAY][SUPPORT] != 0:
            print(doc_name.center(100, '-'))
            print("--Text--")
            for i, line in enumerate(pet_reader.get_doc_sentences(doc_name)):
                print(i, ' '.join(line))
            print()
            print("--Results--:", doc_dict[XOR_GATEWAY])
            print()
            print("--Extracted--:", keywords_gold_token_cls[doc_name][XOR_GATEWAY])
            print()
            print("--Gold standard--:", token_cls_goldstandard[doc_name][XOR_GATEWAY])
            print()

### b) In how many different documents appear the Gateways?

In [31]:
goldstandard_object_path = "data/other/token_goldstandard.pkl"
with open(os.path.abspath(goldstandard_object_path), 'rb') as file:
    goldstandard = pickle.load(file)

In [32]:
def get_gateway_stats(classifications, gateway_type):
    gateway_stats = {}
    for doc_name, doc_dict in classifications.items():
        for gateway_token in doc_dict[gateway_type]:
            if gateway_token not in gateway_stats:
                gateway_stats[gateway_token] = {"count": 1, "docs": 1, "doc_list": set([doc_name])}
            else:
                gateway_stats[gateway_token]["count"] += 1
                gateway_stats[gateway_token]["doc_list"].add(doc_name)
                gateway_stats[gateway_token]["docs"] = len(gateway_stats[gateway_token]["doc_list"])
            
    gateway_stats = collections.OrderedDict(sorted(gateway_stats.items(), 
                                                   key=lambda kv_pair: (kv_pair[1]["count"], kv_pair[1]["docs"]),
                                                   reverse=True))
    return gateway_stats

In [33]:
# XOR GOLDSTANDARD
goldstandard_xor_gateway_stats = get_gateway_stats(goldstandard, XOR_GATEWAY)
for gateway, stats in goldstandard_xor_gateway_stats.items():
    print(gateway.ljust(15), str(stats)[:100])

If              {'count': 52, 'docs': 23, 'doc_list': {'doc-9.1', 'doc-5.3', 'doc-1.1', 'doc-5.1', 'doc-3.8', 'doc-9
or              {'count': 20, 'docs': 15, 'doc_list': {'doc-10.12', 'doc-10.4', 'doc-10.10', 'doc-2.2', 'doc-10.7', 
case            {'count': 12, 'docs': 3, 'doc_list': {'doc-9.5', 'doc-2.1', 'doc-2.2'}}
if              {'count': 9, 'docs': 7, 'doc_list': {'doc-9.5', 'doc-4.1', 'doc-10.2', 'doc-9.1', 'doc-6.1', 'doc-1.
In              {'count': 9, 'docs': 3, 'doc_list': {'doc-9.5', 'doc-2.1', 'doc-2.2'}}
Otherwise       {'count': 6, 'docs': 5, 'doc_list': {'doc-9.5', 'doc-3.5', 'doc-7.1', 'doc-3.6', 'doc-3.8'}}
otherwise       {'count': 6, 'docs': 5, 'doc_list': {'doc-8.3', 'doc-9.1', 'doc-2.1', 'doc-3.2', 'doc-1.2'}}
of              {'count': 5, 'docs': 2, 'doc_list': {'doc-9.5', 'doc-2.2'}}
For             {'count': 5, 'docs': 2, 'doc_list': {'doc-2.1', 'doc-4.1'}}
the             {'count': 4, 'docs': 2, 'doc_list': {'doc-2.1', 'doc-2.2'}}
Sometimes       {'count': 2,

In [34]:
# XOR GOLD
gold_xor_gateway_stats = get_gateway_stats(keywords_gold_token_cls, XOR_GATEWAY)
for gateway, stats in gold_xor_gateway_stats.items():
    print(gateway.ljust(15), str(stats)[:100])

or              {'count': 64, 'docs': 29, 'doc_list': {'doc-10.12', 'doc-10.4', 'doc-10.10', 'doc-5.3', 'doc-10.9', 
for             {'count': 59, 'docs': 23, 'doc_list': {'doc-5.3', 'doc-5.1', 'doc-1.1', 'doc-9.3', 'doc-9.5', 'doc-9
If              {'count': 55, 'docs': 24, 'doc_list': {'doc-10.10', 'doc-9.1', 'doc-5.3', 'doc-5.1', 'doc-1.1', 'doc
case            {'count': 15, 'docs': 4, 'doc_list': {'doc-9.5', 'doc-6.4', 'doc-2.1', 'doc-2.2'}}
if              {'count': 12, 'docs': 10, 'doc_list': {'doc-9.5', 'doc-4.1', 'doc-10.2', 'doc-9.1', 'doc-5.4', 'doc-
For             {'count': 10, 'docs': 4, 'doc_list': {'doc-5.4', 'doc-4.1', 'doc-2.1', 'doc-2.2'}}
In              {'count': 9, 'docs': 3, 'doc_list': {'doc-9.5', 'doc-2.1', 'doc-2.2'}}
of              {'count': 8, 'docs': 3, 'doc_list': {'doc-9.5', 'doc-6.4', 'doc-2.2'}}
otherwise       {'count': 7, 'docs': 5, 'doc_list': {'doc-8.3', 'doc-9.1', 'doc-2.1', 'doc-3.2', 'doc-1.2'}}
Otherwise       {'count': 6, 'docs': 5, 'doc_list':

In [35]:
# XOR LITERATURE
literature_xor_gateway_stats = get_gateway_stats(keywords_literature_token_cls, XOR_GATEWAY)
for gateway, stats in literature_xor_gateway_stats.items():
    print(gateway.ljust(15), str(stats)[:100])

or              {'count': 64, 'docs': 29, 'doc_list': {'doc-10.12', 'doc-10.4', 'doc-10.10', 'doc-5.3', 'doc-10.9', 
If              {'count': 55, 'docs': 24, 'doc_list': {'doc-10.10', 'doc-9.1', 'doc-5.3', 'doc-1.1', 'doc-5.1', 'doc
whether         {'count': 13, 'docs': 9, 'doc_list': {'doc-10.12', 'doc-4.1', 'doc-2.2', 'doc-9.1', 'doc-3.6', 'doc-
if              {'count': 12, 'docs': 10, 'doc_list': {'doc-9.5', 'doc-4.1', 'doc-10.2', 'doc-9.1', 'doc-5.4', 'doc-
case            {'count': 11, 'docs': 4, 'doc_list': {'doc-9.5', 'doc-6.4', 'doc-2.1', 'doc-2.2'}}
when            {'count': 8, 'docs': 7, 'doc_list': {'doc-1.4', 'doc-5.2', 'doc-4.1', 'doc-5.3', 'doc-6.2', 'doc-8.1
In              {'count': 8, 'docs': 3, 'doc_list': {'doc-9.5', 'doc-2.1', 'doc-2.2'}}
When            {'count': 7, 'docs': 7, 'doc_list': {'doc-8.3', 'doc-3.5', 'doc-3.1', 'doc-6.1', 'doc-3.6', 'doc-8.2
otherwise       {'count': 7, 'docs': 5, 'doc_list': {'doc-8.3', 'doc-9.1', 'doc-2.1', 'doc-3.2', 'doc-1.2'}}
of 

In [36]:
# AND GOLDSTANDARD
goldstandard_and_gateway_stats = get_gateway_stats(goldstandard, AND_GATEWAY)
for gateway, stats in goldstandard_and_gateway_stats.items():
    print(gateway.ljust(15), str(stats)[:100])

the             {'count': 2, 'docs': 2, 'doc_list': {'doc-1.1', 'doc-2.2'}}
meantime        {'count': 2, 'docs': 2, 'doc_list': {'doc-3.2', 'doc-1.1'}}
While           {'count': 2, 'docs': 2, 'doc_list': {'doc-1.4', 'doc-1.3'}}
At              {'count': 1, 'docs': 1, 'doc_list': {'doc-2.2'}}
same            {'count': 1, 'docs': 1, 'doc_list': {'doc-2.2'}}
time            {'count': 1, 'docs': 1, 'doc_list': {'doc-2.2'}}
In              {'count': 1, 'docs': 1, 'doc_list': {'doc-1.1'}}
two             {'count': 1, 'docs': 1, 'doc_list': {'doc-2.1'}}
concurrent      {'count': 1, 'docs': 1, 'doc_list': {'doc-2.1'}}
activities      {'count': 1, 'docs': 1, 'doc_list': {'doc-2.1'}}
are             {'count': 1, 'docs': 1, 'doc_list': {'doc-2.1'}}
triggered       {'count': 1, 'docs': 1, 'doc_list': {'doc-2.1'}}
Meantime        {'count': 1, 'docs': 1, 'doc_list': {'doc-3.5'}}
whereas         {'count': 1, 'docs': 1, 'doc_list': {'doc-1.2'}}


In [37]:
# AND GOLD
gold_and_gateway_stats = get_gateway_stats(keywords_gold_token_cls, AND_GATEWAY)
for gateway, stats in gold_and_gateway_stats.items():
    print(gateway.ljust(15), str(stats)[:100])

whereas         {'count': 2, 'docs': 2, 'doc_list': {'doc-9.5', 'doc-1.2'}}
While           {'count': 2, 'docs': 2, 'doc_list': {'doc-1.4', 'doc-1.3'}}
the             {'count': 2, 'docs': 2, 'doc_list': {'doc-1.1', 'doc-2.2'}}
meantime        {'count': 2, 'docs': 2, 'doc_list': {'doc-3.2', 'doc-1.1'}}
Meantime        {'count': 1, 'docs': 1, 'doc_list': {'doc-3.5'}}
At              {'count': 1, 'docs': 1, 'doc_list': {'doc-2.2'}}
same            {'count': 1, 'docs': 1, 'doc_list': {'doc-2.2'}}
time            {'count': 1, 'docs': 1, 'doc_list': {'doc-2.2'}}
In              {'count': 1, 'docs': 1, 'doc_list': {'doc-1.1'}}
two             {'count': 1, 'docs': 1, 'doc_list': {'doc-2.1'}}
concurrent      {'count': 1, 'docs': 1, 'doc_list': {'doc-2.1'}}
activities      {'count': 1, 'docs': 1, 'doc_list': {'doc-2.1'}}
are             {'count': 1, 'docs': 1, 'doc_list': {'doc-2.1'}}
triggered       {'count': 1, 'docs': 1, 'doc_list': {'doc-2.1'}}


In [38]:
# AND LITERATURE
literature_and_gateway_stats = get_gateway_stats(keywords_literature_token_cls, AND_GATEWAY)
for gateway, stats in literature_and_gateway_stats.items():
    print(gateway.ljust(15), str(stats)[:100])

whereas         {'count': 2, 'docs': 2, 'doc_list': {'doc-9.5', 'doc-1.2'}}
While           {'count': 2, 'docs': 2, 'doc_list': {'doc-1.4', 'doc-1.3'}}
In              {'count': 2, 'docs': 2, 'doc_list': {'doc-1.1', 'doc-1.4'}}
the             {'count': 2, 'docs': 2, 'doc_list': {'doc-1.1', 'doc-2.2'}}
meantime        {'count': 2, 'docs': 2, 'doc_list': {'doc-3.2', 'doc-1.1'}}
addition        {'count': 2, 'docs': 1, 'doc_list': {'doc-1.4'}}
to              {'count': 2, 'docs': 1, 'doc_list': {'doc-1.4'}}
Meantime        {'count': 1, 'docs': 1, 'doc_list': {'doc-3.5'}}
in              {'count': 1, 'docs': 1, 'doc_list': {'doc-1.4'}}
At              {'count': 1, 'docs': 1, 'doc_list': {'doc-2.2'}}
same            {'count': 1, 'docs': 1, 'doc_list': {'doc-2.2'}}
time            {'count': 1, 'docs': 1, 'doc_list': {'doc-2.2'}}


### c) Analyze False Positives

In [39]:
def analyze_keyword_metrics(gold_standard, extracted_tokens, gateway_type: str, doc_names=[]):
    keyword_metrics = {}
    empty_stats_dict = {"TP": 0, "FP": 0, "FN": 0, "TPs": [], "FPs": [], "FNs": []}

    for doc_name, doc_dict in keywords_gold_token_cls_results.items():
        
        if doc_name.startswith('doc'):  # result dict contains as well keys for statistics            
            if not doc_names or doc_name in doc_names:  # filter for (optionally) passed list

                # remove gateways step by step if they were identified; left overs are false positives
                not_discovered_golds = [g.lower() for g in gold_standard[doc_name][gateway_type].copy()]

                for extracted in extracted_tokens[doc_name][gateway_type]:
                    extracted = extracted.lower()

                    # setup keyword dict in case it was not observed yet
                    if extracted not in keyword_metrics:
                        keyword_metrics[extracted] = copy.deepcopy(empty_stats_dict)

                    # 1) CHECK FOR FPs
                    if extracted not in not_discovered_golds:
                        keyword_metrics[extracted]["FP"] += 1
                        keyword_metrics[extracted]["FPs"].append(doc_name)

                    # 2) CHECK FOR TPs
                    else:
                        keyword_metrics[extracted]["TP"] += 1
                        keyword_metrics[extracted]["TPs"].append(doc_name)
                        not_discovered_golds.remove(extracted)

                # 3) FILL FNs (FPs from list not_discovered_xor_golds got removed during previous loop)
                for not_extracted in not_discovered_golds:

                    # setup keyword dict in case it was not observed yet
                    if not_extracted not in keyword_metrics:
                        keyword_metrics[not_extracted] = copy.deepcopy(empty_stats_dict)

                    keyword_metrics[not_extracted]["FN"] += 1
                    keyword_metrics[not_extracted]["FNs"].append(doc_name)
    return keyword_metrics


def print_keyword_dict(keyword_metrics, first_x=None, order_by=None):
    
    if order_by:
        keyword_metrics = collections.OrderedDict(sorted(keyword_metrics.items(), key=lambda kv_pair: kv_pair[1][order_by],
                                                         reverse=True))
    for keyword, metrics in keyword_metrics.items():
        print(f"{keyword} ".ljust(15), end='')
        for k, v in metrics.items():
            if (len(k) == 3 and k.endswith("s")) or k == 'doc_list':
                if first_x:
                    print(f"{k}: ", end='')
                    print(list(v)[:first_x], "...", end='')
                    print(", ", end='')
            else:
                print(f"{k}: ", end='')
                print(v, end='')
                print(", ", end='')
        print()


gold_xor_keyword_metrics = analyze_keyword_metrics(token_cls_goldstandard, keywords_gold_token_cls, XOR_GATEWAY)
gold_and_keyword_metrics = analyze_keyword_metrics(token_cls_goldstandard, keywords_gold_token_cls, AND_GATEWAY)
literature_xor_keyword_metrics = analyze_keyword_metrics(token_cls_goldstandard, keywords_literature_token_cls, XOR_GATEWAY)
literature_and_keyword_metrics = analyze_keyword_metrics(token_cls_goldstandard, keywords_literature_token_cls, AND_GATEWAY)

#### EXCLUSIVE

In [40]:
# GOLD keywords
print_keyword_dict(gold_xor_keyword_metrics, first_x=2, order_by="FP")

for            TP: 6, FP: 63, FN: 0, TPs: ['doc-2.1', 'doc-2.1'] ..., FPs: ['doc-3.6', 'doc-2.2'] ..., FNs: [] ..., 
or             TP: 20, FP: 44, FN: 0, TPs: ['doc-2.2', 'doc-1.1'] ..., FPs: ['doc-3.6', 'doc-2.2'] ..., FNs: [] ..., 
if             TP: 61, FP: 6, FN: 0, TPs: ['doc-3.6', 'doc-3.6'] ..., FPs: ['doc-2.1', 'doc-10.10'] ..., FNs: [] ..., 
should         TP: 2, FP: 6, FN: 0, TPs: ['doc-6.1', 'doc-6.1'] ..., FPs: ['doc-2.2', 'doc-2.2'] ..., FNs: [] ..., 
either         TP: 1, FP: 4, FN: 0, TPs: ['doc-2.2'] ..., FPs: ['doc-2.2', 'doc-2.1'] ..., FNs: [] ..., 
in             TP: 9, FP: 3, FN: 0, TPs: ['doc-2.2', 'doc-2.2'] ..., FPs: ['doc-2.2', 'doc-2.2'] ..., FNs: [] ..., 
case           TP: 12, FP: 3, FN: 0, TPs: ['doc-2.2', 'doc-2.2'] ..., FPs: ['doc-2.2', 'doc-2.2'] ..., FNs: [] ..., 
of             TP: 5, FP: 3, FN: 0, TPs: ['doc-2.2', 'doc-2.2'] ..., FPs: ['doc-2.2', 'doc-2.2'] ..., FNs: [] ..., 
otherwise      TP: 12, FP: 1, FN: 0, TPs: ['doc-3.6', 'doc-3.6'] ..., FPs: [

In [41]:
# LITERATURE keywords
print_keyword_dict(literature_xor_keyword_metrics, first_x=2, order_by="FP")

or             TP: 20, FP: 44, FN: 0, TPs: ['doc-2.2', 'doc-1.1'] ..., FPs: ['doc-3.6', 'doc-2.2'] ..., FNs: [] ..., 
when           TP: 0, FP: 15, FN: 0, TPs: [] ..., FPs: ['doc-3.6', 'doc-6.2'] ..., FNs: [] ..., 
whether        TP: 0, FP: 13, FN: 0, TPs: [] ..., FPs: ['doc-3.6', 'doc-2.2'] ..., FNs: [] ..., 
if             TP: 61, FP: 6, FN: 0, TPs: ['doc-3.6', 'doc-3.6'] ..., FPs: ['doc-2.1', 'doc-10.10'] ..., FNs: [] ..., 
either         TP: 1, FP: 4, FN: 0, TPs: ['doc-2.2'] ..., FPs: ['doc-2.2', 'doc-2.1'] ..., FNs: [] ..., 
in             TP: 9, FP: 2, FN: 0, TPs: ['doc-2.2', 'doc-2.2'] ..., FPs: ['doc-2.2', 'doc-6.4'] ..., FNs: [] ..., 
case           TP: 9, FP: 2, FN: 3, TPs: ['doc-2.2', 'doc-2.2'] ..., FPs: ['doc-2.2', 'doc-6.4'] ..., FNs: ['doc-2.1', 'doc-2.1'] ..., 
of             TP: 5, FP: 2, FN: 0, TPs: ['doc-2.2', 'doc-2.2'] ..., FPs: ['doc-2.2', 'doc-6.4'] ..., FNs: [] ..., 
only           TP: 0, FP: 2, FN: 0, TPs: [] ..., FPs: ['doc-2.2', 'doc-4.1'] ..., FNs: [] ..., 


#### PARALLEL

In [42]:
# GOLD keywords
print_keyword_dict(gold_and_keyword_metrics, first_x=2, order_by="FP")

whereas        TP: 1, FP: 1, FN: 0, TPs: ['doc-1.2'] ..., FPs: ['doc-9.5'] ..., FNs: [] ..., 
at             TP: 1, FP: 0, FN: 0, TPs: ['doc-2.2'] ..., FPs: [] ..., FNs: [] ..., 
the            TP: 2, FP: 0, FN: 0, TPs: ['doc-2.2', 'doc-1.1'] ..., FPs: [] ..., FNs: [] ..., 
same           TP: 1, FP: 0, FN: 0, TPs: ['doc-2.2'] ..., FPs: [] ..., FNs: [] ..., 
time           TP: 1, FP: 0, FN: 0, TPs: ['doc-2.2'] ..., FPs: [] ..., FNs: [] ..., 
in             TP: 1, FP: 0, FN: 0, TPs: ['doc-1.1'] ..., FPs: [] ..., FNs: [] ..., 
meantime       TP: 3, FP: 0, FN: 0, TPs: ['doc-1.1', 'doc-3.5'] ..., FPs: [] ..., FNs: [] ..., 
two            TP: 1, FP: 0, FN: 0, TPs: ['doc-2.1'] ..., FPs: [] ..., FNs: [] ..., 
concurrent     TP: 1, FP: 0, FN: 0, TPs: ['doc-2.1'] ..., FPs: [] ..., FNs: [] ..., 
activities     TP: 1, FP: 0, FN: 0, TPs: ['doc-2.1'] ..., FPs: [] ..., FNs: [] ..., 
are            TP: 1, FP: 0, FN: 0, TPs: ['doc-2.1'] ..., FPs: [] ..., FNs: [] ..., 
triggered      TP: 1, FP: 0, FN: 0

In [43]:
# LITERATURE keywords
print_keyword_dict(literature_and_keyword_metrics, first_x=2, order_by="FP")

in             TP: 1, FP: 2, FN: 0, TPs: ['doc-1.1'] ..., FPs: ['doc-1.4', 'doc-1.4'] ..., FNs: [] ..., 
addition       TP: 0, FP: 2, FN: 0, TPs: [] ..., FPs: ['doc-1.4', 'doc-1.4'] ..., FNs: [] ..., 
to             TP: 0, FP: 2, FN: 0, TPs: [] ..., FPs: ['doc-1.4', 'doc-1.4'] ..., FNs: [] ..., 
whereas        TP: 1, FP: 1, FN: 0, TPs: ['doc-1.2'] ..., FPs: ['doc-9.5'] ..., FNs: [] ..., 
at             TP: 1, FP: 0, FN: 0, TPs: ['doc-2.2'] ..., FPs: [] ..., FNs: [] ..., 
the            TP: 2, FP: 0, FN: 0, TPs: ['doc-2.2', 'doc-1.1'] ..., FPs: [] ..., FNs: [] ..., 
same           TP: 1, FP: 0, FN: 0, TPs: ['doc-2.2'] ..., FPs: [] ..., FNs: [] ..., 
time           TP: 1, FP: 0, FN: 0, TPs: ['doc-2.2'] ..., FPs: [] ..., FNs: [] ..., 
meantime       TP: 3, FP: 0, FN: 0, TPs: ['doc-1.1', 'doc-3.5'] ..., FPs: [] ..., FNs: [] ..., 
two            TP: 0, FP: 0, FN: 1, TPs: [] ..., FPs: [] ..., FNs: ['doc-2.1'] ..., 
concurrent     TP: 0, FP: 0, FN: 1, TPs: [] ..., FPs: [] ..., FNs: ['doc-2.1'

### d) Combined analysis (counts & FPs)

In [44]:
# available stats data:
goldstandard_xor_gateway_stats
goldstandard_and_gateway_stats
gold_xor_gateway_stats
gold_and_gateway_stats
literature_xor_gateway_stats
literature_and_gateway_stats

# available metrics data
gold_xor_keyword_metrics
gold_and_keyword_metrics
literature_xor_keyword_metrics
literature_and_keyword_metrics

print()

In [47]:
def merge_stats_metrics_dict(metrics, stats):
    merged = {}
    for gateway_token, gateway_token_stats in stats.items():
        gateway_token = gateway_token.lower()
        if gateway_token in metrics:
            merged[gateway_token] = {**gateway_token_stats, **metrics[gateway_token]}
        else:
            print(gateway_token)
            merged[gateway_token] = gateway_token_stats
    return merged

gold_xor_merged = merge_stats_metrics_dict(gold_xor_keyword_metrics, gold_xor_gateway_stats)
gold_and_merged = merge_stats_metrics_dict(gold_and_keyword_metrics, gold_and_gateway_stats)
literature_xor_merged = merge_stats_metrics_dict(literature_xor_keyword_metrics, literature_xor_gateway_stats)
literature_and_merged = merge_stats_metrics_dict(literature_and_keyword_metrics, literature_and_gateway_stats)

In [66]:
def save_keyword_dict(keyword_dictionary, path, reference_literature_set=None):
    keyword_dictionary = copy.deepcopy(keyword_dictionary)
    # prepare list of dicts (one for each row) for convert to df
    output_list = []
    for keyword, d in keyword_dictionary.items():
        del d["doc_list"]
#         del d["TPs"]
#         del d["FPs"]
#         del d["FNs"]
        d["keyword"] = keyword
        if reference_literature_set:
            d["in_literature"] = keyword in reference_literature_set
        output_list.append(d)
    df = pd.DataFrame(output_list)
    # reorder columns
    cols = df.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    df = df[cols]
    # save as excel
    df.to_excel(path, index=False)

#### 1 - Literature

##### xor

In [56]:
print_keyword_dict(literature_xor_merged, first_x=0, order_by='FP')
save_keyword_dict(literature_xor_merged, "data/keywords/analysis/literature_xor_merged.xlsx")

or             count: 64, docs: 29, TP: 20, FP: 44, FN: 0, 
when           count: 7, docs: 7, TP: 0, FP: 15, FN: 0, 
whether        count: 13, docs: 9, TP: 0, FP: 13, FN: 0, 
if             count: 12, docs: 10, TP: 61, FP: 6, FN: 0, 
either         count: 1, docs: 1, TP: 1, FP: 4, FN: 0, 
case           count: 11, docs: 4, TP: 9, FP: 2, FN: 3, 
in             count: 3, docs: 2, TP: 9, FP: 2, FN: 0, 
of             count: 7, docs: 3, TP: 5, FP: 2, FN: 0, 
only           count: 2, docs: 2, TP: 0, FP: 2, FN: 0, 
otherwise      count: 6, docs: 5, TP: 12, FP: 1, FN: 0, 
until          count: 1, docs: 1, TP: 0, FP: 1, FN: 0, 
not            count: 1, docs: 1, TP: 0, FP: 1, FN: 0, 
unless         count: 1, docs: 1, TP: 0, FP: 1, FN: 0, 


##### and

In [57]:
print_keyword_dict(literature_and_merged, first_x=0, order_by='FP')
save_keyword_dict(literature_and_merged, "data/keywords/analysis/literature_and_merged.xlsx")

in             count: 1, docs: 1, TP: 1, FP: 2, FN: 0, 
addition       count: 2, docs: 1, TP: 0, FP: 2, FN: 0, 
to             count: 2, docs: 1, TP: 0, FP: 2, FN: 0, 
whereas        count: 2, docs: 2, TP: 1, FP: 1, FN: 0, 
while          count: 2, docs: 2, TP: 2, FP: 0, FN: 0, 
the            count: 2, docs: 2, TP: 2, FP: 0, FN: 0, 
meantime       count: 1, docs: 1, TP: 3, FP: 0, FN: 0, 
at             count: 1, docs: 1, TP: 1, FP: 0, FN: 0, 
same           count: 1, docs: 1, TP: 1, FP: 0, FN: 0, 
time           count: 1, docs: 1, TP: 1, FP: 0, FN: 0, 


#### 2 - GOLD

In [70]:
literature_xor_keywords, literature_and_keywords = read_keywords(LITERATURE)
import itertools
literature_xor_keywords_tokens = list(set(itertools.chain(*[keyword.split(" ") for keyword in literature_xor_keywords])))
literature_and_keywords_tokens = list(set(itertools.chain(*[keyword.split(" ") for keyword in literature_and_keywords])))

INFO:utilities:Load keywords 'literature' ...
INFO:utilities:Loaded 15 XOR and 11 AND keywords (literature)
INFO:utilities:Used XOR keywords: ['either', 'else', 'if', 'if not', 'in case', 'in case of', 'only', 'only if', 'or', 'otherwise', 'till', 'unless', 'until', 'when', 'whether']
INFO:utilities:Used AND keywords: ['at the same time', 'concurrently', 'in addition to', 'in parallel', 'in parallel with this', 'in the meantime', 'meantime', 'meanwhile', 'simultaneously', 'whereas', 'while']


##### xor

In [67]:
print_keyword_dict(gold_xor_merged, first_x=0, order_by='FP')
save_keyword_dict(gold_xor_merged, "data/keywords/analysis/gold_xor_merged.xlsx", reference_literature_set=literature_xor_keywords_tokens)

for            count: 10, docs: 4, TP: 6, FP: 63, FN: 0, 
or             count: 64, docs: 29, TP: 20, FP: 44, FN: 0, 
if             count: 12, docs: 10, TP: 61, FP: 6, FN: 0, 
should         count: 2, docs: 1, TP: 2, FP: 6, FN: 0, 
either         count: 1, docs: 1, TP: 1, FP: 4, FN: 0, 
case           count: 15, docs: 4, TP: 12, FP: 3, FN: 0, 
in             count: 3, docs: 2, TP: 9, FP: 3, FN: 0, 
of             count: 8, docs: 3, TP: 5, FP: 3, FN: 0, 
otherwise      count: 6, docs: 5, TP: 12, FP: 1, FN: 0, 
sometimes      count: 2, docs: 1, TP: 4, FP: 1, FN: 0, 
whereas        count: 2, docs: 2, TP: 1, FP: 1, FN: 0, 
the            count: 4, docs: 2, TP: 4, FP: 0, FN: 0, 
it             count: 1, docs: 1, TP: 1, FP: 0, FN: 0, 
can            count: 1, docs: 1, TP: 1, FP: 0, FN: 0, 
also           count: 1, docs: 1, TP: 1, FP: 0, FN: 0, 
happen         count: 1, docs: 1, TP: 1, FP: 0, FN: 0, 
that           count: 1, docs: 1, TP: 1, FP: 0, FN: 0, 
under          count: 1, docs: 1, TP

##### and

In [71]:
print_keyword_dict(gold_and_merged, first_x=0, order_by='FP')
save_keyword_dict(gold_and_merged, "data/keywords/analysis/gold_and_merged.xlsx", reference_literature_set=literature_and_keywords_tokens)

whereas        count: 2, docs: 2, TP: 1, FP: 1, FN: 0, 
while          count: 2, docs: 2, TP: 2, FP: 0, FN: 0, 
the            count: 2, docs: 2, TP: 2, FP: 0, FN: 0, 
meantime       count: 1, docs: 1, TP: 3, FP: 0, FN: 0, 
at             count: 1, docs: 1, TP: 1, FP: 0, FN: 0, 
same           count: 1, docs: 1, TP: 1, FP: 0, FN: 0, 
time           count: 1, docs: 1, TP: 1, FP: 0, FN: 0, 
in             count: 1, docs: 1, TP: 1, FP: 0, FN: 0, 
two            count: 1, docs: 1, TP: 1, FP: 0, FN: 0, 
concurrent     count: 1, docs: 1, TP: 1, FP: 0, FN: 0, 
activities     count: 1, docs: 1, TP: 1, FP: 0, FN: 0, 
are            count: 1, docs: 1, TP: 1, FP: 0, FN: 0, 
triggered      count: 1, docs: 1, TP: 1, FP: 0, FN: 0, 
